In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Example: Reading a CSV file
df = spark.read.csv("test.csv")
df.show()

+---------+-----------+----------+---------+
|      _c0|        _c1|       _c2|      _c3|
+---------+-----------+----------+---------+
| Username| Identifier|First name|Last name|
| booker12|       9012|    Rachel|   Booker|
|   grey07|       2070|     Laura|     Grey|
|johnson81|       4081|     Craig|  Johnson|
|jenkins46|       9346|      Mary|  Jenkins|
|  smith79|       5079|     Jamie|    Smith|
+---------+-----------+----------+---------+

